In [1]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [2]:

data = pd.read_csv('C:/Users/Rhea/Documents/INNOVATION/PROJECT/Amazon_Phone_Blu2_Reviews.csv',
                   names=['Phone_Name','Date','Rating_Out_of_5','Review_Title','Review_Text'])



In [3]:
data.head()

,Phone_Name,Date,Rating_Out_of_5,Review_Title,Review_Text
0,Phone_Name,Date,Rating_Out_of_5,Review_Title,Review_Text
1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",20-Feb-17,5,ABOUT FREAKING TIME!!!! UPDATED POST!!,"Over the years, I've tried maybe 6 Blu phones...."
2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",7-Mar-17,5,Open Honest Review,"People love to get a new phone and, while rid..."
3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",30-May-17,1,Update - don't waste your money! A pretty sol...,Update seven months in: I've had to go from f...
4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",3-Mar-17,1,Spyware still. Come on Blu get it together!,"Still has a security issue,waiting on Blu to r..."


In [4]:
data.shape

(781, 5)

In [5]:
data['Review_Text'].values[5]

"The first time it hit the ground the screen cracked.  I contacted BLU support who told me that there are no replacement screens available for this model.  The screen protector is the only thing holding it together.Included in the box is a 5v 2.0a charger, micoUSB cable, USB-OTG cable, earbuds with microphone, rubber phone case, screen protector, and one of those professional paper clips for opening the sim/SD card tray.  And, there was a screen protector already on the phone.The sim card / SD card tray is pretty flimsy, I wouldn't open it very often.  As other commentators noted, you can either have 2 sim cards or 1 sim card and 1 sd card.  You cannot have all three.The phone case does not fit correctly.  The lower right hand corner does not fully extend over the front edge of the phone, and it feels like the phone will slip out if I grab it by the case.  The case partially covers the microphone hole.There were a few Amazon apps pre-installed, but they had obviously been done so outsi

In [6]:
data_samples=data['Review_Text'].tolist()
data_samples[9]

'Everything works great only downside is the earpiece speaker is quieter than my previous Note 4. I work in an office setting so its quiet in here even then the earpiece speaker is hard to hear unless pressed firmly against your ear.'

In [7]:
data_samples = [str(w).lower() for w in data_samples]

In [8]:
tokenized_docs = [word_tokenize(doc) for doc in data_samples]
print(tokenized_docs[5])

['the', 'first', 'time', 'it', 'hit', 'the', 'ground', 'the', 'screen', 'cracked', '.', 'i', 'contacted', 'blu', 'support', 'who', 'told', 'me', 'that', 'there', 'are', 'no', 'replacement', 'screens', 'available', 'for', 'this', 'model', '.', 'the', 'screen', 'protector', 'is', 'the', 'only', 'thing', 'holding', 'it', 'together.included', 'in', 'the', 'box', 'is', 'a', '5v', '2.0a', 'charger', ',', 'micousb', 'cable', ',', 'usb-otg', 'cable', ',', 'earbuds', 'with', 'microphone', ',', 'rubber', 'phone', 'case', ',', 'screen', 'protector', ',', 'and', 'one', 'of', 'those', 'professional', 'paper', 'clips', 'for', 'opening', 'the', 'sim/sd', 'card', 'tray', '.', 'and', ',', 'there', 'was', 'a', 'screen', 'protector', 'already', 'on', 'the', 'phone.the', 'sim', 'card', '/', 'sd', 'card', 'tray', 'is', 'pretty', 'flimsy', ',', 'i', 'would', "n't", 'open', 'it', 'very', 'often', '.', 'as', 'other', 'commentators', 'noted', ',', 'you', 'can', 'either', 'have', '2', 'sim', 'cards', 'or', '1',

## Removing punctuation
Punctuation can help with tokenizers, but once you've done that, there's no reason to keep it around. There are tons of ways to remove punctuation. Since we have already learned regex, how would we do this?

In [9]:
print(string.punctuation)
str1=string.punctuation
str1=str1.replace('/','')
print(str1)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&'()*+,-.:;<=>?@[\]^_`{|}~


In [10]:
regex = re.compile('[%s]' % re.escape(str1)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    
print (tokenized_docs_no_punctuation[5])

['the', 'first', 'time', 'it', 'hit', 'the', 'ground', 'the', 'screen', 'cracked', 'i', 'contacted', 'blu', 'support', 'who', 'told', 'me', 'that', 'there', 'are', 'no', 'replacement', 'screens', 'available', 'for', 'this', 'model', 'the', 'screen', 'protector', 'is', 'the', 'only', 'thing', 'holding', 'it', 'togetherincluded', 'in', 'the', 'box', 'is', 'a', '5v', '20a', 'charger', 'micousb', 'cable', 'usbotg', 'cable', 'earbuds', 'with', 'microphone', 'rubber', 'phone', 'case', 'screen', 'protector', 'and', 'one', 'of', 'those', 'professional', 'paper', 'clips', 'for', 'opening', 'the', 'sim/sd', 'card', 'tray', 'and', 'there', 'was', 'a', 'screen', 'protector', 'already', 'on', 'the', 'phonethe', 'sim', 'card', '/', 'sd', 'card', 'tray', 'is', 'pretty', 'flimsy', 'i', 'would', 'nt', 'open', 'it', 'very', 'often', 'as', 'other', 'commentators', 'noted', 'you', 'can', 'either', 'have', '2', 'sim', 'cards', 'or', '1', 'sim', 'card', 'and', '1', 'sd', 'card', 'you', 'can', 'not', 'have',

## Cleaning text of stopwords
There are some really basic words that just don't matter. NLTK comes with a list of them for many languages.

In [11]:
tokenized_docs_no_stopwords = []
for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    tokenized_docs_no_stopwords.append(new_term_vector)
            
print (tokenized_docs_no_stopwords[1])

['years', 'tried', 'maybe', '6', 'blu', 'phones', 'looked', 'amazing', 'laggy', 'point', 'drove', 'nuts', 'take', 'back', 'got', 'vivo', 'xl2', 'absolutely', 'happy', 'phone', 'freaking', 'stylish', 'usual', 'tons', 'bells', 'whistles', 'like', 'expensive', 'phone', 'far', 'works', 'amazingly', 'well', 'ca', 'nt', 'believe', 'nice', 'phone', 'price', 'unreal', 'ignore', '64', 'gig', 'sd', 'card', 'limitation', 'took', '128', 'gig', 'problem', 'love', 'new', 'phone', 'blu', 'keeps', 'keep', 'buying', 'blui', 'week', 'still', 'love', 'got', 'better', 'failed', 'mention', 'quick', 'charge', 'comes', 'case', 'arrives', 'screen', 'protector', 'phone', 'bubbles', 'junk', 'film', 'also', 'comes', 'extra', 'screen', 'protector', 'ca', 'nt', 'say', 'much', 'love', 'phone', 'price', 'paid', 'happy']


### Stemming and Lemmatizing
If you have taken linguistics, you may be familiar with morphology. This is the belief that words have a root form. If you want to get to the basic term meaning of the word, you can try applying a stemmer or lemmatizer. Here are three very popular methods ready to go right out of the NLTK box. It's up to you to see which one you want to use.

In [12]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []

for doc in tokenized_docs_no_stopwords:
    final_doc = []
    for word in doc:
        final_doc.append(wordnet.lemmatize(word))
        #final_doc.append(snowball.stem(word))
        #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
    preprocessed_docs.append(final_doc)

print (preprocessed_docs[1])

['year', 'tried', 'maybe', '6', 'blu', 'phone', 'looked', 'amazing', 'laggy', 'point', 'drove', 'nut', 'take', 'back', 'got', 'vivo', 'xl2', 'absolutely', 'happy', 'phone', 'freaking', 'stylish', 'usual', 'ton', 'bell', 'whistle', 'like', 'expensive', 'phone', 'far', 'work', 'amazingly', 'well', 'ca', 'nt', 'believe', 'nice', 'phone', 'price', 'unreal', 'ignore', '64', 'gig', 'sd', 'card', 'limitation', 'took', '128', 'gig', 'problem', 'love', 'new', 'phone', 'blu', 'keep', 'keep', 'buying', 'blui', 'week', 'still', 'love', 'got', 'better', 'failed', 'mention', 'quick', 'charge', 'come', 'case', 'arrives', 'screen', 'protector', 'phone', 'bubble', 'junk', 'film', 'also', 'come', 'extra', 'screen', 'protector', 'ca', 'nt', 'say', 'much', 'love', 'phone', 'price', 'paid', 'happy']


In [13]:
pos_tags=[]

for i in preprocessed_docs:
    pos_tags.append(nltk.pos_tag(i)) 

In [14]:
pos_tags[1]

[('year', 'NN'),
 ('tried', 'VBD'),
 ('maybe', 'RB'),
 ('6', 'CD'),
 ('blu', 'NN'),
 ('phone', 'NN'),
 ('looked', 'VBD'),
 ('amazing', 'JJ'),
 ('laggy', 'JJ'),
 ('point', 'NN'),
 ('drove', 'VBD'),
 ('nut', 'NNS'),
 ('take', 'VB'),
 ('back', 'RB'),
 ('got', 'JJ'),
 ('vivo', 'NN'),
 ('xl2', 'NNP'),
 ('absolutely', 'RB'),
 ('happy', 'JJ'),
 ('phone', 'NN'),
 ('freaking', 'VBG'),
 ('stylish', 'JJ'),
 ('usual', 'JJ'),
 ('ton', 'NN'),
 ('bell', 'NN'),
 ('whistle', 'IN'),
 ('like', 'IN'),
 ('expensive', 'JJ'),
 ('phone', 'NN'),
 ('far', 'RB'),
 ('work', 'NN'),
 ('amazingly', 'RB'),
 ('well', 'RB'),
 ('ca', 'MD'),
 ('nt', 'VB'),
 ('believe', 'VB'),
 ('nice', 'JJ'),
 ('phone', 'NN'),
 ('price', 'NN'),
 ('unreal', 'NN'),
 ('ignore', 'IN'),
 ('64', 'CD'),
 ('gig', 'NNS'),
 ('sd', 'JJ'),
 ('card', 'NN'),
 ('limitation', 'NN'),
 ('took', 'VBD'),
 ('128', 'CD'),
 ('gig', 'NN'),
 ('problem', 'NN'),
 ('love', 'VB'),
 ('new', 'JJ'),
 ('phone', 'NN'),
 ('blu', 'NN'),
 ('keep', 'VB'),
 ('keep', 'JJ'),
 (

In [15]:
noun_words=[]
verb_words=[]
adj_words=[]
for doc in pos_tags:
    for word_pos in doc :
        if word_pos[1] in ('NN','NNS','NNP','NNPS'):
            noun_words.append(word_pos[0]) 
        elif word_pos[1] in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
            verb_words.append(word_pos[0])
        elif word_pos[1] in ('JJR','JJS','JJ'):
            adj_words.append(word_pos[0])
        

In [16]:
import operator


In [17]:
noun_dict=dict((i, noun_words.count(i)) for i in noun_words)
noun_dict = sorted(noun_dict.items(), key=operator.itemgetter(1),reverse=True)
print((noun_dict))

[('phone', 1452), ('blu', 249), ('price', 145), ('month', 141), ('screen', 137), ('work', 130), ('time', 128), ('card', 118), ('use', 116), ('battery', 107), ('camera', 103), ('case', 97), ('problem', 96), ('day', 81), ('vivo', 79), ('nt', 75), ('issue', 75), ('quality', 66), ('product', 66), ('charge', 63), ('year', 62), ('thing', 61), ('memory', 60), ('apps', 60), ('money', 59), ('service', 58), ('everything', 55), ('lot', 54), ('call', 51), ('look', 47), ('protector', 45), ('device', 44), ('love', 44), ('get', 43), ('sim', 43), ('review', 42), ('life', 42), ('feature', 41), ('picture', 41), ('xl', 41), ('samsung', 39), ('xl2', 36), ('week', 36), ('storage', 36), ('customer', 36), ('hour', 35), ('purchase', 34), ('charger', 33), ('way', 33), ('iphone', 31), ('return', 31), ('app', 30), ('cell', 29), ('support', 28), ('brand', 27), ('nothing', 26), ('people', 25), ('deal', 25), ('color', 25), ('replacement', 25), ('reason', 25), ('speaker', 24), ('ram', 24), ('slot', 24), ('button', 2

In [18]:
verb_dict=dict((i, verb_words.count(i)) for i in verb_words)
verb_dict = sorted(verb_dict.items(), key=operator.itemgetter(1),reverse=True)
print((verb_dict))

[('nt', 112), ('get', 95), ('love', 79), ('got', 78), ('bought', 77), ('using', 59), ('go', 55), ('buy', 54), ('working', 54), ('used', 51), ('come', 50), ('say', 50), ('take', 49), ('charging', 46), ('started', 46), ('came', 42), ('make', 38), ('know', 37), ('want', 36), ('getting', 34), ('keep', 33), ('need', 33), ('worked', 30), ('recommend', 30), ('going', 30), ('received', 29), ('think', 28), ('purchased', 27), ('went', 26), ('took', 25), ('made', 24), ('stopped', 24), ('put', 23), ('found', 23), ('give', 23), ('looking', 23), ('blu', 22), ('dropped', 22), ('told', 21), ('see', 21), ('loved', 21), ('expected', 20), ('find', 20), ('tried', 19), ('wanted', 19), ('seems', 19), ('needed', 19), ('said', 18), ('use', 18), ('owned', 18), ('amazing', 16), ('done', 16), ('contacted', 16), ('fixed', 16), ('replace', 16), ('turn', 16), ('set', 16), ('paid', 15), ('decided', 15), ('read', 15), ('running', 14), ('coming', 14), ('trying', 14), ('send', 14), ('expect', 14), ('called', 14), ('see

In [19]:
adj_dict=dict((i, adj_words.count(i)) for i in adj_words)
adj_dict = sorted(adj_dict.items(), key=operator.itemgetter(1),reverse=True)
print((adj_dict))

[('great', 231), ('good', 217), ('nt', 108), ('nice', 83), ('screen', 81), ('last', 71), ('new', 62), ('blu', 48), ('happy', 46), ('little', 43), ('update', 42), ('much', 39), ('sim', 39), ('bad', 38), ('second', 34), ('best', 32), ('better', 32), ('big', 30), ('sd', 29), ('first', 29), ('excellent', 29), ('apps', 29), ('many', 28), ('easy', 28), ('different', 26), ('old', 25), ('android', 25), ('several', 25), ('able', 24), ('previous', 21), ('hard', 20), ('overall', 20), ('awesome', 20), ('internal', 20), ('high', 19), ('sure', 19), ('extra', 18), ('slow', 18), ('decent', 18), ('perfect', 18), ('least', 18), ('cheap', 18), ('original', 18), ('worth', 18), ('touch', 17), ('clear', 17), ('beautiful', 16), ('fine', 16), ('long', 16), ('fast', 16), ('expensive', 15), ('real', 15), ('le', 15), ('poor', 15), ('open', 14), ('come', 14), ('usb', 14), ('full', 14), ('terrible', 14), ('wrong', 14), ('low', 14), ('enough', 13), ('available', 13), ('blue', 13), ('huge', 13), ('right', 13), ('ok'

In [20]:
imp_nouns=[]
imp_verbs=[]
imp_adjs=[]
for k,v in noun_dict:
    if(v>=5):
        imp_nouns.append(k)
for k,v in verb_dict:
    if(v>=5):
        imp_verbs.append(k)
for k,v in adj_dict:
    if(v>=5):
        imp_adjs.append(k)
        

In [21]:
print(len(imp_nouns),len(imp_verbs),len(imp_adjs))

345 190 169


In [22]:
#data_samples=preprocessed_docs

In [23]:
#data_samples=data['Review_Text'].tolist()

In [24]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2,
                                   stop_words='english')

Extracting tf-idf features for NMF...


In [25]:
lemmatised_docs=[]
for i in preprocessed_docs:
    str=' '.join(i)
    lemmatised_docs.append(str)
    

In [26]:
lemmatised_docs[5]

'first time hit ground screen cracked contacted blu support told replacement screen available model screen protector thing holding togetherincluded box 5v 20a charger micousb cable usbotg cable earbuds microphone rubber phone case screen protector one professional paper clip opening sim/sd card tray screen protector already phonethe sim card / sd card tray pretty flimsy would nt open often commentator noted either 2 sim card 1 sim card 1 sd card threethe phone case fit correctly lower right hand corner fully extend front edge phone feel like phone slip grab case case partially cover microphone holethere amazon apps preinstalled obviously done outside google play store would update without asking user modify app installation permission luckily able fully uninstall disabling nt want installing basic google apps say used 11 gb 32 gb luckily phone partitioned main storage multiple segment simulate sd card like blu studio energythe display stunning good contrast color bright curved edge nea

In [27]:
print(type(data_samples[1]),type(lemmatised_docs[1]))

<class 'str'> <class 'str'>


In [28]:
tfidf = tfidf_vectorizer.fit_transform(lemmatised_docs)

In [29]:
tfidf.shape

(781, 1573)

In [30]:
tfidf_features=tfidf_vectorizer.get_feature_names()
len(tfidf_features)

1573

In [31]:
tfidf_features[:20]

['10',
 '100',
 '100150',
 '11',
 '12',
 '128',
 '13mp',
 '14',
 '149',
 '14999',
 '15',
 '150',
 '1520',
 '16g',
 '16gb',
 '17',
 '1st',
 '20',
 '200',
 '2017']

In [32]:
print(tfidf.shape)
# shape of the tfidf is 26,105 means 26 docs(reviews) and 105 features(words) with max_df=0.90, min_df=2

(781, 1573)


In [33]:
tfidf.getrow(0)

<1x1573 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

https://buhrmann.github.io/tfidf-analysis.html

In [34]:
def top_tfidf_feats(row, features,top_n):
    #top_n=25
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [35]:
def top_feats_in_doc(Xtr, features, row_id,top_n):
    #top_n=25
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features,top_n)

In [36]:
def top_mean_feats(Xtr, features,top_n):
    grp_ids=None
    min_tfidf=0.1
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features,top_n)

In [37]:
def top_feats_by_class(Xtr, y, features,top_n):
    min_tfidf=0.1
    
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [38]:
top_feats_in_doc(tfidf,tfidf_features,5,5)

,feature,tfidf
0,card,0.352999
1,sd,0.235409
2,screen,0.230299
3,sim,0.211622
4,case,0.201713


In [39]:
top_mean_feats(tfidf,tfidf_features,50)

,feature,tfidf
0,phone,0.100730
1,good,0.044614
2,great,0.044003
3,love,0.039623
4,nt,0.031653
5,price,0.030379
6,blu,0.029667
7,nice,0.023631
8,work,0.023241
9,like,0.021069


In [40]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                               stop_words='english')

Extracting tf features for LDA...


In [41]:
tf = tf_vectorizer.fit_transform(data_samples)

In [42]:
tf_features=tf_vectorizer.get_feature_names()
len(tf_features)

1723

In [43]:
tf_vectorizer.get_feature_names()[:20]

['00',
 '01',
 '10',
 '100',
 '11',
 '12',
 '128',
 '13mp',
 '14',
 '149',
 '15',
 '150',
 '16g',
 '16gb',
 '17',
 '1st',
 '20',
 '200',
 '2017',
 '2018']

In [44]:
print(tf.shape)

(781, 1723)


In [45]:
tf.data

array([1, 1, 1, ..., 1, 1, 2], dtype=int64)